<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/main/notebooks/Module_4_HYDRAFloods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipyleaflet==0.18.2 geemap hydrafloods

INFO: pip is looking at multiple versions of geemap to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.9 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=257f47d1b8728dc21ee450a59b971075269d110857ee55f779bc4590924f92d3
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e1

Now that we have installed our packages, it's time to import them via `import` so we can use them in this notebook. Below you can find a brief description of what each package does.

* The `geemap` package will allow us to visualize a Google Maps interface interactively within this notebook.
* The `ee` package will allow us to run Earth Engine functions via the python programming language.

In [2]:
import hydrafloods as hf
import geemap
import geemap.colormaps as cm
import ee

Now, we have to authenticate and initialize earth engine. After you run the code below, click through the pop-up window to login to the Google Account associated with your Google Earth Engine account. Click "Continue" until you have returned to this notebook and a green checkmark appears to the left of the code cell below.

If the below cell does not run, try clicking runtime --> Disconnect and Delete Runtime, then click "Connect" in the upper right hand corner. Then rerun the first couple cells in the notebook. You may have to do this multiple times.

If you get an error when running the below cell that says "EEException, you do not have access to the project "rami-for-itc", you need to redo the steps in module 1 as you are not yet added to the cloud project.

In [3]:
my_gee_project = 'servir-sco-assets'
my_gee_folder = 'users/mickymags/flood_intercomparison_morocco_09_09/'
my_projection = 'EPSG:32633'
my_date = "2024-09-26"
my_day_after = "2024-09-27"

In [4]:
ee.Authenticate()

ee.Initialize(project = my_gee_project)

In [5]:
#parent_directory = "projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study"
#parent_directory = "users/mickymags/flood_intercomparison_chad_09_26/"
aoi = ee.FeatureCollection(my_gee_folder + 'aoi')

roi = aoi.geometry()

aoi_centroid = aoi.geometry().centroid()             # Get the center of the AOI
lon = aoi_centroid.coordinates().get(0).getInfo()    # Extract the longitude from the centroid
lat = aoi_centroid.coordinates().get(1).getInfo()    # Extract the latitude from the centroid

In [ ]:
#roi = ee.Geometry.Polygon([[point1, point2, point3, point4]])

In [6]:
from hydrafloods import corrections

In [7]:
s1 = hf.Sentinel1(roi, my_date, my_day_after)

In [8]:
_ = geemap.Map()

In [9]:
hf.__version__

'2023.10.14'

In [10]:
num_imgs = s1.n_images
print('Number of images in the dataset:', num_imgs)

Number of images in the dataset: 6


# Terrain Correction

In [11]:
elv = ee.Image("JAXA/ALOS/AW3D30/V2_2").select("AVE_DSM")

s1_speckle = s1.apply_func(hf.gamma_map)

s1_tc = s1_speckle.apply_func(corrections.slope_correction, elevation=elv, buffer=30)

In [12]:
s1_med = s1_tc.collection.median().clip(aoi)

In [13]:
bmax = hf.bmax_otsu(
    s1_med,
    band = 'VV',
    region = roi,
    initial_threshold = -20,
    thresh_no_data = -16,
    scale = 300,
    bmax_threshold = 0.75
)

In [14]:
edge = hf.edge_otsu(
    s1_med,
    band = 'VV',
    region = roi,
    edge_buffer = 300,
    initial_threshold = -20,
    thresh_no_data = -15,
    scale = 30
).clip(roi)

In [15]:
water_viz = {
    "min": 0,
    "max": 1,
    "palette": ['D3D3D3', '000080']
}

s1_viz = {
    'bands': ['VV'],
    'min': -25,
    'max': 0
}

In [16]:
hydrafloods_default = hf.edge_otsu(
    s1_med,
    band = 'VV',
    region = roi,
    #edge_buffer = 100,
    #initial_threshold = 0,
    thresh_no_data = -0.2,
    #scale = 90
).clip(roi)

In [ ]:
'''
edge_low_thresh = hf.edge_otsu(
    s1_med,
    band = 'VV',
    region = roi,
    edge_buffer = 200,
    initial_threshold = -20,
    thresh_no_data = -0.2,
    #scale = 30
).clip(roi)
'''

"\nedge_low_thresh = hf.edge_otsu(\n    s1_med,\n    band = 'VV',\n    region = roi,\n    edge_buffer = 200,\n    initial_threshold = -20,\n    thresh_no_data = -0.2,\n    #scale = 30\n).clip(roi)\n"

In [ ]:
'''
edge_med_thresh = hf.edge_otsu(
    s1_med,
    band = 'VV',
    region = roi,
    edge_buffer = 200,
    initial_threshold = -15,
    thresh_no_data = -0.2,
    #scale = 30
).clip(roi)
'''

"\nedge_med_thresh = hf.edge_otsu(\n    s1_med,\n    band = 'VV',\n    region = roi,\n    edge_buffer = 200,\n    initial_threshold = -15,\n    thresh_no_data = -0.2,\n    #scale = 30\n).clip(roi)\n"

In [18]:
Map = geemap.Map(center = (lat, lon), zoom = 7)
Map.addLayer(s1_med, s1_viz, 'Sentinel-1')
#Map.addLayer(edge_low_thresh, water_viz, 'Edge -20')
#Map.addLayer(edge_med_thresh, water_viz, 'Edge -18')
Map.addLayer(hydrafloods_default, water_viz, 'HYDRAFloods Default')
Map.addLayerControl()
Map

Map(center=[11.603838129040273, 18.61979504880998], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Define a function that exports an Image to a Google Earth Engine Asset.
def img_exporter(img, asset_id): #scale,

  desc = 'Flood_Map_Export_'
  region_ = aoi.geometry()
  geemap.ee_export_image_to_asset(image = img,
                                  assetId = 'users/mickymags/flood_intercomparison/chad_09_26_take2/hydrafloods', #my_gee_folder + 'hydrafloods,
                                  description = desc,
                                  region = region_,
                                  crs = my_projection,
                                  #scale = scale_,
                                  maxPixels = 1e13)
  return 0

In [ ]:
img_exporter(hydrafloods_default, 'hydrafloods_mosaic')

0